In [1]:
import os, sys, torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

# Add parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from multilayer_model.m3_multi_bce import M3_Multi_BCE
from multilayer_model.m3_multi_mse import M3_Multi_MSE
from multilayer_loss.l_multi3_final_logic import l_multi3_final_loss
from vae_earlystopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 800
print(f"Using device: {device}")

Using device: cuda


In [2]:
# 1. Data Loading
x1_raw = np.load('../data/metal.npy')
x2_raw = np.load('../data/support_norm.npy')
x3_raw = np.load('../data/pre_fin.npy')
c_raw = np.load('../data/re_fin.npy')

# Data Splitting (8:1:1)
indices = np.arange(len(x1_raw))
tr_idx, te_idx = train_test_split(indices, test_size=0.1, random_state=42)
tr_idx, va_idx = train_test_split(tr_idx, test_size=0.1, random_state=42)

sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()

def prep(data, idx, scaler, fit=False):
    subset = data[idx]
    return scaler.fit_transform(subset) if fit else scaler.transform(subset)

x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

def to_t(arr): return torch.tensor(arr, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]
c_dim = c_tr.shape[1]

In [ ]:
def train_specialist(model, mode='mse'):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    es = EarlyStopping(patience=40)
    
    for epoch in range(1, epochs + 1):
        model.train()
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            optimizer.zero_grad()
            preds, mus, lvs = model(b1, b2, b3, bc)
            loss = l_multi3_final_loss(preds, [b1, b2, b3], mus, lvs, epoch, epochs, mode)
            loss.backward()
            optimizer.step()
        
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vp, vm, vv = model(v1, v2, v3, vc)
                v_loss += l_multi3_final_loss(vp, [v1, v2, v3], vm, vv, epoch, epochs, mode).item()
        
        avg_v_loss = v_loss / len(val_loader)
        if epoch % 50 == 0: 
            print(f"[{mode.upper()}] Epoch {epoch} | Val Loss: {avg_v_loss:.6f}")
        
        if es(avg_v_loss, model):
            print(f"Early stopping at epoch {epoch}")
            break
    
    es.load_best_model(model)
    return model

print("Stage 1: Training Specialist BCE Model...")
model_bce = train_specialist(M3_Multi_BCE(x_dims, c_dim).to(device), mode='bce')

print("\nStage 2: Training Specialist MSE Model (Focus on Last Layer)... ")
model_mse = train_specialist(M3_Multi_MSE(x_dims, c_dim).to(device), mode='mse')

Stage 1: Training Specialist BCE Model...


In [ ]:
model_bce.eval()
model_mse.eval()
all_gen_preds, all_trues = [], []

print("🚀 Starting Pure Generation Test (Condition-based Sampling)...")

with torch.no_grad():
    for b1, b2, b3, bc in test_loader:
        bc = bc.to(device)
        
        # 1. BCE 모델: 존재 확률 생성 (Prior Sampling)
        # z를 인코더에서 뽑지 않고 표준정규분포에서 직접 샘플링
        b_gen = model_bce.generate(bc, device)
        prob3 = torch.sigmoid(b_gen[2]) # Pretreatment 확률
        
        # 2. MSE 모델: 수치 함량 생성 (Prior Sampling)
        m_gen = model_mse.generate(bc, device)
        val3 = m_gen[2] # Pretreatment 수치
        
        # 3. Gated 결합: 확률 * 수치
        # 생성 모드에서는 두 모델의 잠재적 의도가 결합되어 노이즈가 제거됨
        combined = (prob3 * val3).cpu().numpy()
        
        # 4. 역스케일링 및 데이터 수집
        preds_inv = sc3.inverse_transform(combined)
        trues_inv = sc3.inverse_transform(b3.cpu().numpy())

        all_gen_preds.append(preds_inv)
        all_trues.append(trues_inv)

y_true = np.concatenate(all_trues).flatten()
y_pred = np.concatenate(all_gen_preds).flatten()

# 결과 출력
final_r2 = r2_score(y_true, y_pred)
print(f"\n🏆 Generation-based R2 Score (Pretreatment): {final_r2:.4f}")

# 5. 시각화 (예측값 튐 현상 확인)
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.scatter(y_true, y_pred, alpha=0.4, color='dodgerblue', label='Generated Samples')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2, label='Perfect Match')
plt.title("Generation Verification: Predicted vs Actual")
plt.xlabel("Actual Pretreatment Value"); plt.ylabel("Generated Value")
plt.legend(); plt.grid(True); plt.show()